In [4]:
!pip install folium
!pip install pandas


In [5]:
import folium 
import pandas as pd

In [6]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [7]:
pip install wget


Note: you may need to restart the kernel to use updated packages.


In [8]:
import wget


In [9]:
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [37]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_df = launch_df[['Launch Site', 'Lat', 'Long']]
launch_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [12]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [13]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [14]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

In [15]:
launch_sites_dict = launch_df.set_index('Launch Site').T.to_dict('list')
launch_sites_dict

{'CCAFS LC-40': [28.56230197, -80.57735648],
 'CCAFS SLC-40': [28.56319718, -80.57682003],
 'KSC LC-39A': [28.57325457, -80.64689529],
 'VAFB SLC-4E': [34.63283416, -120.6107455]}

In [27]:
from folium import DivIcon

# Define launch site coordinates
launch_sites = {
    'CCAFS LC-40': [28.56230197, -80.57735648],
    'CCAFS SLC-40': [28.56319718, -80.57682003],
    'KSC LC-39A': [28.57325457, -80.64689529],
    'VAFB SLC-4E': [34.63283416, -120.6107455]
}

# Initialize a Folium map centered at the first launch site
site_map = folium.Map(location=[28.56230197, -80.57735648], zoom_start=5)

# Loop through the launch sites and add a circle and marker for each
for site, coordinate in launch_sites.items():
    # Add a circle for the launch site
    folium.Circle(
        location=coordinate,      # Coordinates of the site
        radius=1000,              # Radius in meters
        color='#000000',          # Circle border color
        fill=True,                # Fill the circle
        fill_color='#3186cc',     # Fill color
        popup=folium.Popup(site)  # Popup with the site's name
    ).add_to(site_map)

    # Add a marker for the launch site
    folium.map.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20, 20),  # Icon size
            icon_anchor=(0, 0),  # Anchor position
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site  # Custom HTML for the marker label
        )
    ).add_to(site_map)
    

In [29]:
site_map

In [31]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [33]:
marker_cluster = MarkerCluster()

In [41]:
def assign_marker_color(launch_outcome):
    if launch_outcome == 1 :
        return 'green'
    else :
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [55]:
for index, row in spacex_df.iterrows():
    # Extract data for each launch
    site = row['Launch Site']
    lat = row['Lat']
    long = row['Long']
    classes = row['class']

    # Add a marker to the cluster
    folium.Marker(
        location=[lat, long],
        popup=f"Launch Site: {site}<br>Outcome: {classes}",
        icon=folium.Icon(color="green" if classes == "1" else "red")
    ).add_to(marker_cluster)

# Add the marker cluster to the map
site_map.add_child(marker_cluster)

# Display the map
site_map